<a href="https://colab.research.google.com/github/greymouse1/ss_lab1/blob/main/ss_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import numpy for matrix operations
import numpy as np
# import math for advanced operations
import math

Task #1

In [27]:
# define known values (everything is in meters and dgm)

# GRS80 parameters
a = 6378137
f = 1/298.257222101


# semi-minor axis
b = a*(1-f)

# ellipsoid height
h = 85

# eccentricity
e_sq = ((a**2)-(b**2))/(a**2)

# Lat
lat_d = 59
lat_m = 20
lat_s = 59

# Lon
lon_d = 18
lon_m = 4
lon_s = 10

# define functions for conversion from dgm to radians

def dgm_rad (d,m,s) :
  decimal_degrees = d + m/60 + s/3600
  print(f"Decimal degrees are {decimal_degrees}")
  degrees_radians = decimal_degrees * (math.pi / 180)
  return degrees_radians

# Lat in rads
lat_r = dgm_rad(lat_d,lat_m,lat_s)

# Long in rads
lon_r = dgm_rad(lon_d,lon_m,lon_s)

N = a / math.sqrt(1 - e_sq * math.sin(lat_r)**2)

# define functions for conversion of geodetic to cartesian coordinates
X = (N+h)*(math.cos(lat_r)*math.cos(lon_r))
Y = (N+h)*(math.cos(lat_r)*math.sin(lon_r))
Z = (N*(1-e_sq)+h)*math.sin(lat_r)

print(f"X coordinate is {X}")
print(f"Y coordinate is {Y}")
print(f"Z coordinate is {Z}")

# define auxiliary values
p = math.sqrt(X**2 + Y**2)

teta = math.atan(Z / (p*math.sqrt(1 - e_sq)))

# define functions for conversion from cartesian to geodetic

lon = math.degrees(math.atan(Y/X))
lat = math.degrees(math.atan((Z + ((a*e_sq)/math.sqrt(1-e_sq))*math.sin(teta)**3) / (p - a*e_sq*math.cos(teta)**3) ))
h_reverse = (p / math.cos(math.radians(lat))) - N

print(f"Longitude is {lon}")
print(f"Latitude is {lat}")
print(f"h is {h_reverse}")

Decimal degrees are 59.349722222222226
Decimal degrees are 18.069444444444443
X coordinate is 3098917.2417289796
Y coordinate is 1011053.4113269701
Z coordinate is 5463972.352679947
Longitude is 18.069444444444443
Latitude is 59.34972222222222
h is 84.99999999906868


Task #2

In [57]:
# satellite positions file path
satellites = "/content/drive/MyDrive/Colab Notebooks/satellites.txt"

# Load data into a NumPy array using space as the delimiter
data = np.genfromtxt(satellites, delimiter=' ')  # Load the data

# Create a dictionary with point names as keys and vectors as values
sat_dict = {}

for row in data:
    sat_name = row[0] # Convert bytes to string for the point name
    sat_coordinates = np.array(row[1:3], dtype=float)  # Create a NumPy array for coordinates
    sat_dict[str(int(sat_name))] = sat_coordinates  # Add to the dictionary

# Printing the dictionary
print("Dictionary with point names as keys and coordinates as values:")
for key, value in sat_dict.items():
    print(f"{key}: {value}")

Dictionary with point names as keys and coordinates as values:
1: [20168.35 16031.5 ]
2: [-13216.25 -15675.2 ]
3: [24273.68 11044.11]
4: [ -5635.61 -24940.09]
5: [-22597.38  12612.97]
6: [-11025.39  11875.83]
7: [-10223.35 -18084.47]
8: [ 4.630000e+00 -2.633901e+04]
9: [-14334.82   5786.63]
10: [-18293.96  -3681.19]
11: [15283.17 -6864.3 ]
13: [  8518.87 -11993.68]
14: [10970.08 17831.95]
15: [ -161.45 26358.93]
16: [14966.51  7382.03]
18: [-14735.48  18485.22]
19: [23960.3   5523.98]
20: [ 22851.63 -13287.33]
21: [-7400.88 22801.14]
22: [-2798.63 17719.89]
23: [19248.21 -4298.33]
24: [  4966.96 -24972.12]
25: [10251.68 22378.96]
26: [-24659.05  -7748.72]
27: [  5091.26 -23132.94]
28: [  4721.79 -18461.06]
29: [-24660.67  -9835.58]
30: [-18247.85  18324.56]
31: [  -764.51 -16615.4 ]


# New section